In [ ]:
from component.CalculoMatrizTempoViagemComponent import CalculoMatrizTempoViagemComponent
from model.constants.ParametrosConstantes import ParametrosConstantes
from service.ModalidadeTransporteService import ModalidadeTransporteService
from service.MunicipioService import MunicipioService
from util.BancoDadosUtil import BancoDadosUtil
from util.DataFrameUtil import DataFrameUtil
from util.LoggerUtil import LoggerUtil
from util.OSUtil import OSUtil
from util.OSMNXUtil import OSMNXUtil
from util.WarningUtil import WarningUtil

from pandas import DataFrame
from time import sleep

In [ ]:
# Configurações iniciais

OSMNXUtil.configurar_osmnx()

WarningUtil.ignorar_warning(categoria=DeprecationWarning)
WarningUtil.ignorar_warning(categoria=UserWarning)

log = LoggerUtil.configurar_logger(arquivo_log=ParametrosConstantes.CAMINHO_LOG_CALCULO_MATRIZ_TEMPO_VIAGEM)

In [ ]:
calculo_matriz_tempo_viagem_component = CalculoMatrizTempoViagemComponent()
modalidade_transporte_service = ModalidadeTransporteService()
municipio_service = MunicipioService()

In [ ]:
while True:
    log.info(msg="Iniciando o cálculo da matriz de tempos de viagem dos municípios.")
    conexao_bd = BancoDadosUtil.estabelecer_conexao_banco()

    if calculo_matriz_tempo_viagem_component.verificar_nao_existencia_registros_pendentes(conexao_bd):
        log.info(msg="Encerrando a conexão com o banco de dados.")
        BancoDadosUtil.encerrar_conexao_banco(conexao_bd)
        
        break
    
    log.info(msg=f"Buscando os próximos {ParametrosConstantes.BATCH_QTDE_REGISTROS_ETAPA_CALCULO_MATRIZ_TEMPO_VIAGEM} municípios a serem processados.")

    gdf_municipio = municipio_service.buscar_registros_pendentes_calculo_matriz_tempo_viagem(conexao_bd)
    df_modalidade_transporte = modalidade_transporte_service.buscar_todos_registros(conexao_bd)

    log.info(msg="Calculando a matriz de tempos de viagem dos municípios.")

    df_resultado = DataFrameUtil.processar_dataframe_dask(
        df=gdf_municipio,
        funcao=calculo_matriz_tempo_viagem_component.processar_particao_dask,
        meta=DataFrame(columns=["codigo_municipio", "lista_dict_matriz_tempo_viagem", "lista_dict_historico_erro", "status"]),
        qtde_particoes=ParametrosConstantes.BATCH_QTDE_PARTICOES_DASK_ETAPA_CALCULO_MATRIZ_TEMPO_VIAGEM,
        df_modalidade_transporte=df_modalidade_transporte,
        conexao_bd=conexao_bd
    )

    calculo_matriz_tempo_viagem_component.persistir_resultado(df_resultado, conexao_bd)

    if ParametrosConstantes.OSMXN_USAR_CACHE and ParametrosConstantes.OSMNX_LIMPAR_CACHE:
        OSUtil.deletar_diretorio(diretorio=ParametrosConstantes.DIRETORIO_CACHE_OSMNX)

    log.info(msg="Encerrando a conexão com o banco de dados.")
    BancoDadosUtil.encerrar_conexao_banco(conexao_bd)

    sleep(10)